In [ ]:
import openai
import requests
import json
import subprocess
import uuid

In [ ]:
# Set your azure OpenAI deployment name of complete model 
# text-davinci-003 corresponds to GPT-3.51
COMPLETIONS_MODEL_ENGINE = "text-davinci-003"

In [ ]:
# set your api key
openai.api_key = "<YOUR_OPENAI_API_KEY>"
# your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_base =  "<YOUR_OPENAI_ENDPOINT>" 
# this may change in the future
openai.api_type = "azure"
# this may change in the future
openai.api_version = "2022-12-01" 
# set your azure search api key
AZURE_SEARCH_API_KEY = "<YOUR_AZURE_SEARCH_API_KEY>"
# set your azure search endpoint should look like the following https://YOUR_RESOURCE_NAME.search.windows.net/
AZURE_SEARCH_ENDPOINT = "<YOUR_AZURE_SEARCH_ENDPOINT>"
# set your azure search indexer name
AZURE_SEARCH_INDEXER_NAME = "<YOUR_AZURE_SEARCH_INDEXER_NAME>"

In [ ]:
# Details refer: https://platform.openai.com/docs/api-reference/completions/create
COMPLETIONS_API_PARAMS = {
    # Controls diversity via nucleus sampling: 0.5 means half of all likelihood-weighted options are considered
    "top_p": 1,
    # Controls randomness: lowering temperature results in less random completions. 
    # At zero the model becomes deterministic
    "temperature": 0.0,
    # How much to penalize new tokens based on their existing frequency in the text so far. 
    # Decreases the model's likelihood to repeat the same line verbatim
    "frequency_penalty" : 0,
    # How much to penalize new tokens based on whether they appear in the text so far. 
    # Increases the model's likelihood to talk about new topics
    "presence_penalty" : 0,
    # The maximum number of tokens to generate in the output. One token is roughly 4 characters
    "max_tokens": 2000,
    "engine": COMPLETIONS_MODEL_ENGINE,
}

In [ ]:
def construct_prompt_python_api(question: str, top: int = 0) -> str:

    header ="#write azure search python sdk with question below"
    header += "\n#question: " + question + ""
    if top > 0:
        header += "\n#select top "+str(top)
    header += "\n#api-key: "+ AZURE_SEARCH_API_KEY
    header += "\n#azure_search_endpint: " + AZURE_SEARCH_ENDPOINT
    header += "\n#azure_search_indexer: " + AZURE_SEARCH_INDEXER_NAME
    header += "\n#api-version: 2021-04-30-Preview"
    header += "\n#API key is not permitted in the URI question string"
    header += "\n#return response.text"
    header += "\n\n"
    return header

def construct_prompt_url(question: str, top: int = 0) -> str:

    header ="#write a url about question below by azure search Lucene question language"
    header += "\n#question: " + question + ""
    if top > 0:
        header += "\n#select top "+str(top)
    header += "\n#azure_search_endpint: " + AZURE_SEARCH_ENDPOINT
    header += "\n#azure_search_indexer: " + AZURE_SEARCH_INDEXER_NAME
    header += "\n#api-version: 2021-04-30-Preview"
    header += "\n#result: <construct the full url>"
    header += "\n\n"
    return header

def construct_prompt_from_question(question: str, prompt_from_search: str) -> str:
    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
    return header + "".join(prompt_from_search) + "\n\n Q: " + question + "\n A:"

def construct_prompt_from_question_summary(prompt_from_search: str) -> str:
    header = """please summary the context below."\n\nContext:\n"""
    return header + "".join(prompt_from_search) + "\n"

In [ ]:
def generate_python_api_from_question(
    question: str,
    top: int = 0,
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt_python_api(
        question,
        top
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

def question_search_content_by_python_api(filename: str, code: str) -> str:

    with open(filename, "w") as f:
        f.write(code)
    
    resp_py = json.loads(subprocess.getoutput("python3.9 "+filename))
    subprocess.getoutput("rm -rf "+filename)
    return resp_py

def generate_url_from_question(
    question: str,
    top: int = 0,
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt_url(
        question,
        top
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

def question_search_content_by_url(url: str) -> str:
    # API version
    api_version = '2021-04-30-Preview'

    # API key
    api_key = AZURE_SEARCH_API_KEY

    # Set the authorization header
    headers = {'api-key': api_key}
    
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.text

def answer_question_from_search_context(
    question: str,
    prompt_from_search: str,
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt_from_question(
        question,
        prompt_from_search
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

def summary_search_context(
    prompt_from_search: str,
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt_from_question_summary(
        prompt_from_search
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

In [ ]:
def query_search_content_by_generate_python_api(question: str, top: int = 0) -> str:
    generate_response_python_api = generate_python_api_from_question(question, top, show_prompt=False)
    print(generate_response_python_api)

    # gererate 
    uid = uuid.uuid4()
    generate_file_name = "search_restful_"+str(uid)+".py"

    question_response_python_api = question_search_content_by_python_api(generate_file_name, generate_response_python_api)
    final_response_python_api = question_response_python_api
    return final_response_python_api

In [ ]:
def query_search_content_by_generate_url(question: str, top: int = 0) -> str:
    generate_response_url = generate_url_from_question(question, top, show_prompt=False)
    print(generate_response_url)

    # question search content by url
    question_response_url = question_search_content_by_url(url=generate_response_url)
    final_response_url = json.loads(question_response_url)
    return final_response_url

In [ ]:
question = "how people describe New York and location is USA"

In [ ]:
final_response_python_api = query_search_content_by_generate_python_api(question, top=2)

In [ ]:
prompt_from_search_python_api = ""
for i in final_response_python_api["value"]:
    prompt_from_search_python_api += i["merged_text"] + "\n"

In [ ]:
answer_question_from_search_context(question, prompt_from_search_python_api, show_prompt=True)

In [ ]:
final_response_url = query_search_content_by_generate_url(question, top=2)

In [ ]:
prompt_from_search_url = ""
for i in final_response_url["value"]:
    prompt_from_search_url += i["merged_text"] + "\n"

In [ ]:
answer_question_from_search_context(question, prompt_from_search_url, show_prompt=True)

In [ ]:
final_response_python_api_summary = query_search_content_by_generate_python_api(question)

In [ ]:
for i in final_response_python_api_summary["value"]:
    i["merged_text"] = summary_search_context(i["merged_text"])
    del i["content"]
    del i["entities"]

final_response_python_api_summary